# Assignment 6

### Part A

In [1]:
import numpy as np
import pandas as pd

Consider a DataFrame giving information about the price of a stock in 1 year, and the probability of that price occurring.

In [2]:
data = pd.DataFrame(data = {"Price in 1 Year" : [32, 43, 54],
                            "Probability" : [0.3, 0.4, 0.3]})
                    
data

Price in 1 Year  Probability
0               32          0.3
1               43          0.4
2               54          0.3

Write the function ```exp_return```, which calculates the expected return of the stock, given the current stock price ```P```.

In [41]:
def exp_return(data, P):
    # Write your code here
    # exp_return(data, 40) should output 0.07500000000000002
    expected_return = 0
    
    for i in range(len(data['Price in 1 Year'])):
        future_price = data['Price in 1 Year'][i]
        probability = data['Probability'][i]
        
        return_percentage = (future_price / P) - 1
        expected_return += return_percentage * probability
        
    return expected_return

#data = {"Price in 1 Year" : [32, 43, 54], "Probability" : [0.3, 0.4, 0.3]}
#print(exp_return(data, 40))

0.07500000000000002


Write the function ```volatility```, which calculates the expected return of the stock, given the current stock price ```P```.

In [42]:
def volatility(data, P):
    # Write your code here
    # volatility(data,40) should output 0.21301408404140795
    expected_return = 0
    var = 0
    
    for i in range(len(data['Price in 1 Year'])):
        future_price = data['Price in 1 Year'][i]
        probability = data['Probability'][i]
        return_percentage = (future_price / P) - 1
        expected_return += return_percentage * probability
        var += (return_percentage**2) * probability
    vol = (var - expected_return**2)**(1/2)
    return vol

#data = {"Price in 1 Year" : [32, 43, 54], "Probability" : [0.3, 0.4, 0.3]}
#print(volatility(data, 40))

0.21301408404140795


Write the function ```interval```, which calculates the 95% confidence interval for the expected return of the data. The corresponding Z score is 1.96. The function should return a list of size 2, with the first element representing the lower bound, and the second element representing the upper bound. 

In [43]:
def interval(data, P):
    # Write your code here
    # interval(data, 40) should output [-0.16604812797447732, 0.3160481279744774]
    expected_return = exp_return(data, P)
    vol = volatility(data,P)
    SE = vol/((len(data['Price in 1 Year']))**(1/2))
    x = expected_return - 1.96*SE
    y = expected_return + 1.96*SE
    return [x,y]
#data = {"Price in 1 Year" : [32, 43, 54], "Probability" : [0.3, 0.4, 0.3]}
#print(interval(data, 40))

[-0.16604812797447732, 0.3160481279744774]


Write the function ```exp_return_capm```, which, given the beta of a security ```beta```, the return on the market ```market_return```,  and the risk free interest rate ```rf```, calculates the expected return of the security.

In [44]:
def exp_return_capm(beta, market_return, rf):
    # Write your code here
    # exp_return_capm(-0.6,0.1, 0.02) should output -0.028
    ER = rf + beta*(market_return-rf)
    return ER
#print(exp_return_capm(-0.6,0.1, 0.02))

-0.028


### Part B

Write the function ```clean_data```, which will take a path to a csv file like the one provided. This function should remove all observations with a non number ```RET```, and remove all securities with (strictly) less than 500 observations. Your returned DataFrame should have 4 (non-index) columns, `PERMNO`, `date`, `CUSIP`, and `RET`.

In [52]:
import numpy as np
import pandas as pd

def clean_data(path):
    # Write your code here
    # This should yield data with 403316 rows × 4 columns
    df = pd.read_csv(path)
    df["RET"]= df["RET"].apply(pd.to_numeric, errors='coerce')
    df = df[pd.notnull(df['RET'])]
    permno_counts = df['PERMNO'].value_counts()
    valid_permnos = permno_counts.index[permno_counts >= 500]
    df = df[df['PERMNO'].isin(valid_permnos)]
    return df

Write the function ```avg_return```, which will calculate the average return for each security. This function should take in a DataFrame ```data```, formatted like one outputted by the function ```clean_data```. The results should be returned in a DataFrame formatted like the one supplied.

Additionally, write the function ```security_volatilities```, which does the same thing as ```avg_return```, except it returns the volatilities of the returns of the securities.

In [53]:
import numpy as np
import pandas as pd

def avg_return(data):
    # Write your code here
    # This should yield data with 679 rows × 2 columns
    avg_return = data.groupby('PERMNO')['RET'].mean().reset_index()
    avg_return.columns = ['PERMNO','Avg_Return']
    
    output = pd.DataFrame(data = {
        "CUSIP" : avg_return['PERMNO'],
        "Average Return" : avg_return['Avg_Return']
    })
    
    return output

#data = clean_data("/Users/hannahong/Downloads/monthlycrsp_mod6.csv")
#print(avg_return(data))

In [54]:
import numpy as np
import pandas as pd

def security_volatilities(data):
    # Write your code here
    # This should yield data with 679 rows × 2 columns
    volatilities = data.groupby('PERMNO')['RET'].std().reset_index()
    volatilities.columns = ['PERMNO','Volatility']
    output = pd.DataFrame(data = {
        "CUSIP" : volatilities['PERMNO'],
        "Volatility" : volatilities['Volatility']
    })
    
    return output

#data = clean_data("/Users/hannahong/Downloads/monthlycrsp_mod6.csv")
#print(security_volatilities(data))

Write the function ```correlation```, which will return the correlation between average return and volatility. This function takes two DataFrames, ```ret_data``` and ```vol_data```, which are the outputs of ```avg_return``` and ```security_volatilities``` respectively.

In [55]:
def correlation(ret_data, vol_data):
    # Write your code here
    # correlation(rets, vols) should yield 0.44837090727059614
    merged_data = pd.merge(ret_data, vol_data, on='CUSIP')
    corr = merged_data['Average Return'].corr(merged_data['Volatility'])
    return corr

#data = clean_data("/Users/hannahong/Downloads/monthlycrsp_mod6.csv")
#ret_data = avg_return(data)
#vol_data = security_volatilities(data)
#correlation(ret_data, vol_data)